### Create the table that store configurations, per stream.

In [1]:
import boto3
from botocore.exceptions import ClientError

dynamodb = boto3.resource("dynamodb", region_name="eu-west-1")
TABLE_NAME = "stream_configs"

try:
    table = dynamodb.create_table(
        TableName=TABLE_NAME,
        KeySchema=[
            {"AttributeName": "device_id", "KeyType": "HASH"},
            {"AttributeName": "config_type", "KeyType": "RANGE"},
        ],
        AttributeDefinitions=[
            {"AttributeName": "device_id", "AttributeType": "S"},
            {"AttributeName": "config_type", "AttributeType": "S"},
        ],
        BillingMode="PAY_PER_REQUEST"
    )
    print("Creating table...")
    table.wait_until_exists()
    print(f"Table '{TABLE_NAME}' created.")
except ClientError as e:
    if e.response["Error"]["Code"] == "ResourceInUseException":
        print(f"Table '{TABLE_NAME}' already exists.")
    else:
        raise

Creating table...
Table 'stream_configs' created.


In [ ]:
import boto3
from datetime import datetime
from decimal import Decimal

dynamodb = boto3.resource("dynamodb", region_name="eu-west-1")
TABLE_NAME = "stream_configs"
table = dynamodb.Table(TABLE_NAME)

def put_ai_config():
    device_id = "B8A44FB3A1F9__front"
    config_type = "ai_config"

    config_data = {
        "debug": True,
        "models": [
            {
                "name": "object_detection_then_ppe",
                "threshold": Decimal(0.5),
                "classes_to_detect": ['person', 'car_plate','bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat',
                    'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat',
                    'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe']
            }
        ]
    }

    item = {
        "device_id": device_id,
        "config_type": config_type,
        "config": config_data,
        "updated_at": datetime.utcnow().isoformat()
    }

    table.put_item(Item=item)
    print(f"✅ Inserted config for device_id='{device_id}', config_type='{config_type}'")

put_ai_config()


✅ Inserted config for device_id='B8A44FB3A1F9__front', config_type='ai_config'
